Data Fusion Pipeline

In [1]:
# paths
from pathlib import Path

# paths to files
CLIN_PATH = Path("/Users/madhurabn/Desktop/adni/data/processed/clinical_cognitive_demographic__imputed.xlsx")
CSF_PATH  = Path("/Users/madhurabn/Desktop/adni/data/processed/csf_imputed.xlsx")
MRI_PATH  = Path("/Users/madhurabn/Desktop/adni/data/processed/mri_final.xlsx")

OUTDIR = Path("/Users/madhurabn/Desktop/adni/data/processed")
OUTDIR.mkdir(parents=True, exist_ok=True)
FUSION_XLSX = OUTDIR / "fusion_master.xlsx"
print("Output will be written to:", FUSION_XLSX)


Output will be written to: /Users/madhurabn/Desktop/adni/data/processed/fusion_master.xlsx


In [2]:

import re
import pandas as pd

def normalize_colnames(cols):
    norm = []
    for c in cols:
        cc = str(c).strip().lower()
        cc = re.sub(r"[^0-9a-zA-Z]+", "_", cc)
        cc = re.sub(r"_+", "_", cc).strip("_")
        norm.append(cc)
    return norm

PTID_TOKENS = ["ptid","rid","subject","subject_id","participant","id"]
DIAG_TOKENS = ["diagnosis","dx","dx_group","group","final_diagnosis"]

def find_exact_col(df, tokens):
    cols = [c for c in df.columns]
    for t in tokens:
        if t in cols:
            return t
    return None

def find_contains_col(df, tokens):
    for c in df.columns:
        lc = c.lower()
        for t in tokens:
            if t in lc:
                return c
    return None


In [3]:
#  Load the imputed data files 
import pandas as pd

def _read_excel_any(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"Not found: {path}")
    df = pd.read_excel(path)
    df.columns = normalize_colnames(df.columns)
    return df

clin = _read_excel_any(CLIN_PATH)
csf  = _read_excel_any(CSF_PATH)
mri  = _read_excel_any(MRI_PATH)

print("Loaded shapes:")
print(" clinical:", clin.shape)
print(" csf     :", csf.shape)
print(" mri     :", mri.shape)

# Drop diagnosis-like columns from CSF and MRI to avoid duplicate labels
DIAG_CANDIDATES = [
    "diagnosis","Diagnosis","DX","DX_bl","DXCHANGE","Clinical_Diagnosis","Diag","diagnosis_bl","DXCURREN"
]

def _drop_diag_cols(df, name):
    drop_these = [c for c in df.columns if c != "PTID" and (c.lower() in [d.lower() for d in DIAG_CANDIDATES] or "diagnos" in c.lower())]
    if drop_these:
        df.drop(columns=drop_these, inplace=True)
        print(f"[{name}] dropped diagnosis-like columns: {drop_these}")
    return df

csf = _drop_diag_cols(csf, "csf")
mri = _drop_diag_cols(mri, "mri")

print("Loaded shapes after dropping:")
print(" clinical:", clin.shape)
print(" csf     :", csf.shape)
print(" mri     :", mri.shape)


Loaded shapes:
 clinical: (1205, 17)
 csf     : (749, 8)
 mri     : (1171, 102)
[mri] dropped diagnosis-like columns: ['diagnosis']
Loaded shapes after dropping:
 clinical: (1205, 17)
 csf     : (749, 8)
 mri     : (1171, 101)


In [4]:
#  Detect keys and target 
ptid_clin = find_exact_col(clin, PTID_TOKENS) or find_contains_col(clin, PTID_TOKENS)
ptid_csf  = find_exact_col(csf,  PTID_TOKENS) or find_contains_col(csf,  PTID_TOKENS)
ptid_mri  = find_exact_col(mri,  PTID_TOKENS) or find_contains_col(mri,  PTID_TOKENS)

diag_col  = find_exact_col(clin, DIAG_TOKENS) or find_contains_col(clin, DIAG_TOKENS)

if not ptid_clin or not ptid_csf or not ptid_mri:
    raise ValueError(f"Could not detect PTID columns. Found -> clinical: {ptid_clin}, csf: {ptid_csf}, mri: {ptid_mri}")
if not diag_col:
    raise ValueError("Could not detect diagnosis column in the clinical file. Please rename it (e.g., 'diagnosis').")

print("Detected columns:")
print(" ptid (clinical):", ptid_clin)
print(" ptid (csf     ):", ptid_csf)
print(" ptid (mri     ):", ptid_mri)
print(" diagnosis     :", diag_col)


Detected columns:
 ptid (clinical): ptid
 ptid (csf     ): ptid
 ptid (mri     ): ptid
 diagnosis     : diagnosis


In [5]:
# Prepare for fusion 
keep_clin  = clin.rename(columns={ptid_clin: "ptid"}).copy()
csf_ren    = csf.rename(columns={ptid_csf:  "ptid"}).copy()
mri_ren    = mri.rename(columns={ptid_mri:  "ptid"}).copy()

if diag_col != "diagnosis":
    keep_clin = keep_clin.rename(columns={diag_col: "diagnosis"})

print("Column counts after normalization:")
print(" clinical:", keep_clin.shape[1])
print(" csf     :", csf_ren.shape[1])
print(" mri     :", mri_ren.shape[1])


Column counts after normalization:
 clinical: 17
 csf     : 8
 mri     : 101


In [6]:
#  Inner-merge on PTID only 
fusion = keep_clin.merge(csf_ren, on="ptid", how="inner", suffixes=("", "_csf"))
fusion = fusion.merge(mri_ren, on="ptid", how="inner", suffixes=("", "_mri"))

print("Fusion shape (inner on PTID across all three):", fusion.shape)
print("Patients overlap:", fusion["ptid"].nunique())


Fusion shape (inner on PTID across all three): (745, 124)
Patients overlap: 745


In [7]:
#  Drop rows with empty diagnosis after merge if any
before = fusion.shape[0]
if "diagnosis" not in fusion.columns:
    raise KeyError("'diagnosis' column missing from fused table. Ensure clinical file contained it.")

fusion = fusion[~fusion["diagnosis"].isna() & (fusion["diagnosis"].astype(str).str.strip() != "")].copy()
after = fusion.shape[0]
print(f"Dropped {before - after} rows due to empty diagnosis.")


Dropped 0 rows due to empty diagnosis.


In [8]:
# summaries
import numpy as np
print("\n Fusion summary ")
print("Rows:", fusion.shape[0], " Columns:", fusion.shape[1])
print("Unique PTIDs:", fusion["ptid"].nunique())
    
if "diagnosis" in fusion.columns:
    print("\nDiagnosis counts:")
    print(fusion["diagnosis"].value_counts(dropna=False))

# Missingness after fusion
miss = fusion.isna().mean().sort_values(ascending=False)
print("\nTop-15 missingness after fusion:")
print(miss.head(15))



 Fusion summary 
Rows: 745  Columns: 124
Unique PTIDs: 745

Diagnosis counts:
diagnosis
MCI         375
DEMENTIA    185
CN          185
Name: count, dtype: int64

Top-15 missingness after fusion:
ptid                                                          0.0
thickness_average_aparc_stats_of_leftprecuneus                0.0
thickness_average_aparc_stats_of_rightrostralmiddlefrontal    0.0
cortical_volume_aparc_stats_of_rightsuperiorfrontal           0.0
thickness_average_aparc_stats_of_rightprecentral              0.0
thickness_average_aparc_stats_of_leftposteriorcingulate       0.0
subcortical_volume_aseg_stats_of_rightchoroidplexus           0.0
thickness_average_aparc_stats_of_rightbankssts                0.0
surface_area_aparc_stats_of_leftinferiortemporal              0.0
subcortical_volume_aseg_stats_of_csf                          0.0
cortical_volume_aparc_stats_of_leftparsorbitalis              0.0
thickness_average_aparc_stats_of_rightparsopercularis         0.0
subcortical

In [9]:
# Drop unwanted columns at the end 
for col in ["visit_csf", "visit_mri", "rid","genotype","rid_mri","visit","RID", "entry_date"]:
    if col in fusion.columns:
        fusion.drop(columns=[col], inplace=True)

# 1) Find the gender/sex column
gender_like = [c for c in fusion.columns if c.lower() in ["gender","sex","ptgender","sex_bl"]]
print("Gender-like columns:", gender_like)
gcol = gender_like[0]  


# after normalization
norm_vals = (fusion[gcol].astype(str).str.strip().str.lower())
print("Unique normalized values (head):", norm_vals.unique()[:20])
print(norm_vals.value_counts().head(10))

# Encode gender to numeric: male=0, female=1
fusion["gender"] = (
    fusion["gender"]
    .astype(str).str.strip().str.lower()
    .map({"male": 0, "female": 1})
    .astype("Int64")   # keeps NaN-safe integers
)

print(fusion["gender"].value_counts(dropna=False))

    
# Save fusion data file 
from pathlib import Path
print(fusion.shape)
FUSION_XLSX.parent.mkdir(parents=True, exist_ok=True)
fusion.to_excel(FUSION_XLSX, index=False)
print("Saved:", FUSION_XLSX)


Gender-like columns: ['gender']
Unique normalized values (head): ['male' 'female']
gender
male      387
female    358
Name: count, dtype: int64
gender
0    387
1    358
Name: count, dtype: Int64
(745, 118)


Saved: /Users/madhurabn/Desktop/adni/data/processed/fusion_master.xlsx
